In [13]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
import joblib


# Define constants
IMG_DIR = 'C:\\Users\\ahmed\\Downloads\\oxford-iiit-pet\\images\\images'  # Path to extracted images
IMG_SIZE = 128
LIMIT_PER_CLASS = 200

cat_breeds = ['Abyssinian', 'Bengal', 'Birman', 'Bombay', 'British_Shorthair',
              'Egyptian_Mau', 'Maine_Coon', 'Persian', 'Ragdoll',
              'Russian_Blue', 'Siamese', 'Sphynx']


# Balanced loading
X, y = [], []
cat_count, dog_count = 0, 0

# Load images (RGB, larger size)
for img_name in tqdm(os.listdir(IMG_DIR)):
    if not img_name.lower().endswith('.jpg'):
        continue

    breed = img_name.split("_")[0]
    label = None
    if breed in cat_breeds and cat_count < LIMIT_PER_CLASS:
        label = 0
        cat_count += 1
    elif breed not in cat_breeds and dog_count < LIMIT_PER_CLASS:
        label = 1
        dog_count += 1

    if label is None:
        continue

    try:
        img_path = os.path.join(IMG_DIR, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img_flat = img.flatten()
        X.append(img_flat)
        y.append(label)
    except:
        continue

X = np.array(X)
y = np.array(y)

print(f"✅ Loaded {len(X)} images.")

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# PCA to reduce feature dimensions
print("📉 Applying PCA...")
pca = PCA(n_components=200)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train SVM with manual hyperparameters
print("🤖 Training SVM...")
clf = SVC(kernel='rbf', C=1.0, gamma='scale')
clf.fit(X_train_pca, y_train)

# Evaluate
y_pred = clf.predict(X_test_pca)
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))
print("✅ Accuracy:", accuracy_score(y_test, y_pred))

# Save models
joblib.dump(clf, "svm_model_pca.pkl")
joblib.dump(pca, "pca_model.pkl")


100%|████████████████████████████████████████████████████████████████████████████| 7393/7393 [00:00<00:00, 9372.26it/s]


✅ Loaded 399 images.
📉 Applying PCA...
🤖 Training SVM...

📊 Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.83      0.86        46
           1       0.79      0.88      0.83        34

    accuracy                           0.85        80
   macro avg       0.85      0.85      0.85        80
weighted avg       0.86      0.85      0.85        80

✅ Accuracy: 0.85


['pca_model.pkl']

In [29]:
import gradio as gr
import cv2
import numpy as np
import joblib

# Load trained model and PCA
model = joblib.load("svm_model_pca.pkl")
pca = joblib.load("pca_model.pkl")

def predict(image):
    try:
        # Resize and preprocess
        image = cv2.resize(image, (128, 128))
        image_flat = image.flatten().reshape(1, -1)

        # Apply PCA
        image_pca = pca.transform(image_flat)

        # Predict
        pred = model.predict(image_pca)[0]
        label = "🐱 Cat" if pred == 0 else "🐶 Dog"
        return label
    except Exception as e:
        return f"Error: {str(e)}"

# Launch app
demo = gr.Interface(fn=predict,
                    inputs=gr.Image(type="numpy"),
                    outputs="text",
                    title="Cat vs Dog Classifier 🐾",
                    description="Upload a pet image to classify it as a cat or dog")

demo.launch()



* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


In [30]:
demo.close()


Closing server running on port: 7867
